In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib inline

In [ ]:
pd.read_excel('../data/CBSA_ZIP_032020.xlsx')

In [ ]:
pd.read_excel('../data/Addendum_B/2020_january_web_addendum_b.12312019.xlsx', header = 2)

In [ ]:
pd.read_csv('../data/Medicare_Provider_Util_Payment_PUF_CY2017/Medicare_Provider_Util_Payment_PUF_CY2017.txt', sep='\t', skiprows = [1], usecols = lambda column : column not in ['average_submitted_chrg_amt', 'average_Medicare_payment_amt', 'average_Medicare_standard_amt', 'hcpcs_drug_indicator', 'medicare_participation_indicator'], low_memory=False)

In [ ]:
pd.read_excel('../data/MUP_OHP_R19_P04_V10_D17_APC_Provider/MUP_OHP_R19_P04_V10_D17_APC_Provider.xlsx', header = 5)